In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  979.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  249.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 5526 personas en los últimos 7 días 

Un positivo PCR cada 3123 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] > 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,979.0,161.0,276.0,510743.0,31.522703,54.038920,18.0
Huelva-Costa,516.0,88.0,143.0,288115.0,30.543359,49.632959,13.0
Huelva (capital),249.0,26.0,46.0,143663.0,18.097910,32.019379,7.0
Isla Cristina,52.0,18.0,34.0,21264.0,84.650113,159.894658,4.0
Aljaraque,54.0,8.0,11.0,21260.0,37.629351,51.740357,2.0
Condado-Campiña,297.0,57.0,105.0,155057.0,36.760675,67.717033,2.0
Sierra de Huelva-Andévalo Central,149.0,10.0,19.0,67571.0,14.799248,28.118572,2.0
Valverde del Camino,26.0,3.0,5.0,12820.0,23.400936,39.001560,1.0
Villalba del Alcor,36.0,9.0,26.0,3338.0,269.622528,778.909527,1.0


In [14]:
if tabla[tabla['Diferencia PCR'] > 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
# Save to csv
# tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)

In [16]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Cabezas Rubias,8.0,6.0,7.0,730.0,821.917808,958.904110,0.0
Villalba del Alcor,36.0,9.0,26.0,3338.0,269.622528,778.909527,1.0
Escacena del Campo,15.0,6.0,11.0,2284.0,262.697023,481.611208,0.0
Zalamea la Real,19.0,5.0,6.0,3068.0,162.972621,195.567145,0.0
Bollullos Par del Condado,35.0,17.0,23.0,14272.0,119.114350,161.154709,0.0
Isla Cristina,52.0,18.0,34.0,21264.0,84.650113,159.894658,4.0
Palma del Condado (La),31.0,6.0,10.0,10761.0,55.756900,92.928167,0.0
Palos de la Frontera,18.0,10.0,10.0,11289.0,88.581805,88.581805,0.0
Condado-Campiña,297.0,57.0,105.0,155057.0,36.760675,67.717033,2.0
